In [2]:
from sensordataset import SensorDataset
data_types = ['Segment Acceleration', 'Segment Angular Velocity', 'Sensor Magnetic Field']
train_dataset = SensorDataset(input_dir='./sensor_Training', data_types=data_types)
val_dataset = SensorDataset(input_dir='./sensor_Validation', data_types=data_types)

import pandas as pd
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers = 4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers = 4)

In [4]:
for data, label in train_loader:
    print("data shape:", data.shape)
    break

data shape: torch.Size([32, 12, 10, 9])


In [38]:
import torch
import torch.nn as nn

class FallDetection1DCNN(nn.Module):
    def __init__(self, num_classes=1):
        super(FallDetection1DCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=9, out_channels=16, kernel_size=3, stride=1, padding=2)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=2)
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=16, kernel_size=3, stride=1, padding=2)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.pool2 = nn.MaxPool1d(kernel_size=3, stride=3)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        batch_size = x.size(0)
        feature_vectors = []

        for i in range(batch_size):
            sample_features = []

            for j in range(x.size(1)):
                sample = x[i, j].unsqueeze(0)
                sample = sample.permute(0, 2, 1)

                out = self.conv1(sample)
                out = self.relu(out)
                out = self.pool1(out)

                out = self.conv2(out)
                out = self.relu(out)
                out = self.pool1(out)

                out = self.conv3(out)
                out = self.relu(out)
                out = self.pool1(out)
                
                pooled_out = torch.mean(out, dim=2)
                sample_features.append(pooled_out)

            concatenated = torch.cat(sample_features, dim=1)
            feature_vectors.append(concatenated)

        feature_vectors = torch.cat(feature_vectors, dim=0)
        feature_vectors = self.pool2(feature_vectors)
        x = self.fc(feature_vectors)
        return torch.sigmoid(x)

In [39]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

model = FallDetection1DCNN().to(device)

In [40]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [41]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

num_epochs = 30 
best_val_loss = float('inf') 
for epoch in range(num_epochs):

    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for samples, labels in train_loader:
        samples, labels = samples.to(device), labels.float().to(device).unsqueeze(1)

        outputs = model(samples)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * samples.size(0) 

        predictions = (outputs >= 0.5).float()  
        correct_train += (predictions == labels).sum().item()
        total_train += labels.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    train_accuracy = correct_train / total_train
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")

    model.eval() 
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    all_val_labels = []
    all_val_predictions = []

    with torch.no_grad(): 
        for val_samples, val_labels in val_loader:
            val_samples, val_labels = val_samples.to(device), val_labels.float().to(device).unsqueeze(1)

            val_outputs = model(val_samples)
            val_loss += criterion(val_outputs, val_labels).item() * val_samples.size(0)

            val_predictions = (val_outputs >= 0.5).float()
            correct_val += (val_predictions == val_labels).sum().item()
            total_val += val_labels.size(0)

            all_val_labels.extend(val_labels.cpu().numpy())
            all_val_predictions.extend(val_predictions.cpu().numpy())

    val_loss /= len(val_loader.dataset)
    val_accuracy = correct_val / total_val
    print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    cm = confusion_matrix(all_val_labels, all_val_predictions)
    precision = precision_score(all_val_labels, all_val_predictions)
    recall = recall_score(all_val_labels, all_val_predictions)
    f1 = f1_score(all_val_labels, all_val_predictions)

    print("Confusion Matrix:\n", cm)
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), './best_model.pth')
        print("Best model updated and saved!")

print("Training complete.")

Epoch [1/50], Training Loss: 0.6196, Training Accuracy: 0.6408
Epoch [1/50], Validation Loss: 0.5296, Validation Accuracy: 0.7500
Confusion Matrix:
 [[  0  71]
 [  0 213]]
Precision: 0.7500, Recall: 1.0000, F1-Score: 0.8571
Best model updated and saved!
Epoch [2/50], Training Loss: 0.5216, Training Accuracy: 0.7493
Epoch [2/50], Validation Loss: 0.4918, Validation Accuracy: 0.7500
Confusion Matrix:
 [[  0  71]
 [  0 213]]
Precision: 0.7500, Recall: 1.0000, F1-Score: 0.8571
Best model updated and saved!
Epoch [3/50], Training Loss: 0.4746, Training Accuracy: 0.7493
Epoch [3/50], Validation Loss: 0.4363, Validation Accuracy: 0.7500
Confusion Matrix:
 [[  0  71]
 [  0 213]]
Precision: 0.7500, Recall: 1.0000, F1-Score: 0.8571
Best model updated and saved!
Epoch [4/50], Training Loss: 0.4210, Training Accuracy: 0.7661
Epoch [4/50], Validation Loss: 0.3903, Validation Accuracy: 0.8134
Confusion Matrix:
 [[ 22  49]
 [  4 209]]
Precision: 0.8101, Recall: 0.9812, F1-Score: 0.8875
Best model upd

KeyboardInterrupt: 